# Метод ближайших соседей

**KNN (k Nearest Neighbor или k Ближайших Соседей) — очень популярный алгоритм машинного обучения с учителем.**

Алгоритм достаточно быстрый, но требует большого объема памяти. 

**KNN может быть применен как для решения задач классификации, так и для решения задачи регрессии.**

**Краткий алгоритм работы KNN:**
1. Пусть имеется набор данных, состоящий из $N$ объектов $X_i(i=1,...n)$ для каждого которых задан класс $C_i$.
2. Выбираем число $K$ - количество соседей. $K$ - любое целое число.
3. Далее вычисляется расстояние от каждого объекта  $X_i$  до каждого из объектов  $X$  обучающей выборки. 
*Методы расчета расстояния: Евклидово, Манхэттенское или Хэмминговское расстояние.*
4. Выбирается  $K$  объектов, расстояние до которых минимально;
5. Далее объект относится к классу, к которому относится большинство из выбранных $K$ ближайших соседей:

$$a(u) = \underset{y}{\text{argmax}}\sum_{i=1}^{k}[y_{u}^{(i)}=y],$$

то есть провести голосование.


Под задачу регрессии метод адаптируется довольно легко – на 5 шаге возвращается не метка, а число – среднее (или медианное) значение целевого признака среди соседей.

**Пример.**

Пусть $K = 5$.

&X_i& - серая точка.

Хотим определить к какому классу она относится.


SimpleKnnExample.png

## Вычислительная сложность алгоритма

Чтобы вычислить вычислительную сложность KNN, давайте рассмотрим $d$ - мерное пространство, $k$ — количество соседей, а $n$ — общее количество точек обучающих данных.

Чтобы понять, как мы можем рассчитать сложность этого алгоритма, взгляните на формальный псевдокод! Каждое вычисление расстояния требует $O(d)$ времени выполнения, поэтому расчет для одной точки до всех других требует $O(nd)$ работы. Для каждой итерации на третьем шаге мы выполняем $O(n)$ работы, перебирая наблюдения обучающего набора, поэтому в целом шаг требует $O(nk)$ работы. Первый шаг требует только $O(n)$ работы, поэтому мы получаем время выполнения $O(nd + kn)$.

# Практика

Датасет: https://www.kaggle.com/uciml/pima-indians-diabetes-database

In [1]:
from google.colab import files
uploaded = files.upload()

In [2]:
import pandas as pd

df = pd.read_csv("diabetes.csv")
df

FileNotFoundError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

sns.countplot(df['Outcome'], ax=ax, palette='hls')
plt.show()

In [ ]:
X = df.drop(['Outcome'], axis = 1)
y = df['Outcome']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

pd.DataFrame(X_st, columns = X.columns)

# X - вектор признаков исходного набора
# X_st - вектор стандартизированных признаков

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

Метод ближайщих соседей, реализован в библиотеке sklearn в модуле neighbors.

Для использования необходимо явным образом указывать сколько соседей необходимо учитывать (если не указывать по умолчанию будет 5)

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

`sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)`

1. `n_neighbors` - количество соседей
2. `weights` - функция для весов:
-  `uniform` - однородные веса. Все точки в каждой окрестности имеют одинаковый вес.
- `distance` - точки веса обратно пропорциональны их расстоянию. в этом случае более близкие соседи точки запроса будут иметь большее влияние, чем соседи, которые находятся дальше.
3. `algorithm` - Алгоритм, используемый для вычисления ближайших соседей.
4. `metric` - метод расчета расстояния (евклидово, манххэтонское и т.д.)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # классификатор
from sklearn.neighbors import KNeighborsRegressor # регрессор

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

# создаем модель
# как подобрать лучшее число соседей?
scores = []
for i in range(1,15):
    score = {}
    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test) # метод predict возвращает вектор ответов для X_test
    
    score['k'] = i
    score['accuracy'] = accuracy_score(answers,y_test)
    score['recall'] = recall_score(answers,y_test)
    score['presicion'] = precision_score(answers,y_test)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

In [ ]:
plt.plot(scores['k'], scores['accuracy'], label='Accuracy')
plt.plot(scores['k'], scores['recall'], label='Recall')
plt.plot(scores['k'], scores['presicion'], label='Presicion')
plt.xlabel('K')
plt.xlabel('Score')
plt.legend()

In [ ]:
knn = KNeighborsClassifier(7)

# обучаем
knn.fit(X_train,y_train)

# предсказываем значения
y_pred = knn.predict(X_test)

confusion_matrix(y_test,y_pred)

# 0 - общее 133+43
# 1 - общее37+43

# Балансировка

В машинном обучении балансировка классов означает балансировку классов с несбалансированными выборками. Перед использованием алгоритма машинного обучения важно избежать дисбаланса классов, потому что наша конечная цель – обучить модель машинного обучения, которая хорошо обобщается для всех возможных классов, предполагая, что у нас есть двоичный набор данных с равным количеством выборок.

Ребалансировка классов может происходить путём увеличения числа примеров миноритарного класса (**undersampling**), либо путём сокращения числа примеров мажоритарного (**oversampling**). Возможно также и сочетание обоих подходов.

**Сокращение числа примеров мажоритарного класса**

В данном случае можно рандомно удалить некоторое число объектов из мажоритарного класса до достижения баланса. Либо использовать существующие алгоритмы: Nearmiss1/2, Tomek links, Edited nearest neighbors (ENN)

`from imblearn.under_sampling import NearMiss`

`from imblearn.under_sampling import RandomUnderSampler`

**Увеличение числа примеров миноритарного класса**

**Дублирование** примеров миноритарного класса (Oversampling). Самый простой метод – это дублирование примеров миноритарного класса. В зависимости от того, какое соотношение классов необходимо получить в выборке, выбирается случайным образом соответствующее количество наблюдений для дублирования.

Такой подход к восстановлению баланса не всегда является наиболее эффективным, поэтому был предложен специальный метод увеличения числа наблюдений миноритарного класса – алгоритм SMOTE (Synthetic Minority Oversampling Technique).

**Алгоритм SMOTE**. В основе алгоритма лежит идея генерации некоторого количества искусственных наблюдений, которые были бы «похожи» на наблюдения, имеющиеся в миноритарном классе, но при этом не дублировали их. 

Подробнее о различных алгоритмах балансировки можно найти в библиотеке `imbalanced-learn`.

https://imbalanced-learn.org/stable/index.html

In [ ]:
X = df.drop(['Outcome'], axis = 1)
y = df['Outcome']

print(f'Features shape: {X.shape}')
print(f'Classes distribution:\n{y.value_counts()}')

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0, k_neighbors=10)

columns = X.columns

os_data_X, os_data_y = os.fit_resample(X, y)
os_data_X = pd.DataFrame(os_data_X, columns = columns)
os_data_y = pd.DataFrame(os_data_y, columns = ['Outcome'])

In [ ]:
print(f'Features shape after SMOTE: {os_data_X.shape}')
print(f'Classes distribution after SMOTE:\n{os_data_y.value_counts()}')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(os_data_X)

pd.DataFrame(X_st, columns = X.columns)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_st, os_data_y, test_size=1/3, random_state=42)

print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # классификатор
from sklearn.neighbors import KNeighborsRegressor # регрессор

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

# создаем модель
# как подобрать лучшее число соседей?
scores = []
for i in range(1,15):
    score = {}
    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test) # метод predict возвращает вектор ответов для X_test
    
    score['k'] = i
    score['accuracy'] = accuracy_score(answers,y_test)
    score['recall'] = recall_score(answers,y_test)
    score['presicion'] = precision_score(answers,y_test)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

In [ ]:
plt.plot(scores['k'], scores['accuracy'], label='Accuracy')
plt.plot(scores['k'], scores['recall'], label='Recall')
plt.plot(scores['k'], scores['presicion'], label='Presicion')
plt.xlabel('K')
plt.xlabel('Score')
plt.legend()

# Задание

https://www.kaggle.com/competitions/hotel-booking-demand-3

In [80]:
from sklearn.neighbors import KNeighborsClassifier # классификатор
from sklearn.neighbors import KNeighborsRegressor # регрессор
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [81]:
df = pd.read_csv("train_final.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44638 entries, 0 to 44637
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           44638 non-null  object 
 1   is_canceled                     44638 non-null  int64  
 2   lead_time                       44638 non-null  int64  
 3   arrival_date_year               44638 non-null  int64  
 4   arrival_date_month              44638 non-null  object 
 5   arrival_date_week_number        44638 non-null  int64  
 6   arrival_date_day_of_month       44638 non-null  int64  
 7   stays_in_weekend_nights         44638 non-null  int64  
 8   stays_in_week_nights            44638 non-null  int64  
 9   adults                          44638 non-null  int64  
 10  children                        44638 non-null  float64
 11  babies                          44638 non-null  int64  
 12  meal                            

In [161]:
df['country'].value_counts()

PRT    16999
GBR     4914
FRA     4207
ESP     3382
DEU     2957
       ...  
MRT        1
DMA        1
LCA        1
ETH        1
CAF        1
Name: country, Length: 156, dtype: int64

In [140]:
X, y = df.drop(columns=["is_canceled", "reservation_status_date", 'country']), df["is_canceled"]

In [141]:
X_test = pd.read_csv("test_final.csv").drop(columns=["reservation_status_date"])

In [143]:
n = 44638

In [144]:
X = pd.concat([X_test, X], axis=0)

In [145]:
X

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,City Hotel,73,2016,July,28,6,0,2,1,0.0,...,0,A,A,0,No Deposit,0,Transient,107.10,0,0
1,City Hotel,37,2015,October,43,24,2,5,1,1.0,...,0,A,A,0,No Deposit,0,Transient,87.78,0,0
2,City Hotel,190,2017,April,14,6,2,3,2,0.0,...,0,A,A,0,No Deposit,0,Transient,88.40,0,0
3,City Hotel,287,2016,August,35,24,1,4,2,0.0,...,0,B,B,1,No Deposit,0,Transient,76.71,0,0
4,Resort Hotel,386,2016,October,43,20,1,3,2,0.0,...,0,A,A,0,No Deposit,0,Transient-Party,49.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44633,City Hotel,31,2017,June,26,27,0,1,1,0.0,...,0,D,D,0,No Deposit,0,Transient,119.00,0,2
44634,Resort Hotel,116,2015,September,39,26,2,5,2,0.0,...,0,E,F,0,No Deposit,0,Transient,93.86,0,0
44635,City Hotel,89,2017,April,14,3,1,2,2,0.0,...,0,A,A,0,No Deposit,0,Transient,99.00,0,1
44636,City Hotel,277,2016,September,37,5,2,5,2,0.0,...,0,A,A,0,No Deposit,0,Transient-Party,89.14,0,1


In [146]:
X = pd.get_dummies(X, columns=["hotel", "arrival_date_month", "meal", "country", "market_segment", "distribution_channel", "reserved_room_type", "assigned_room_type", "deposit_type", "customer_type"])

In [147]:
X

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,73,2016,28,6,0,2,1,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,37,2015,43,24,2,5,1,1.0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,190,2017,14,6,2,3,2,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,287,2016,35,24,1,4,2,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,386,2016,43,20,1,3,2,0.0,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44633,31,2017,26,27,0,1,1,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
44634,116,2015,39,26,2,5,2,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
44635,89,2017,14,3,1,2,2,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
44636,277,2016,37,5,2,5,2,0.0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [148]:
y_train.value_counts()

is_canceled
0              28254
1              11920
dtype: int64

In [149]:
X, _X = X[:n], X[n:]
len(X), len(_X)

(44638, 23525)

In [150]:
os = SMOTE(random_state=0, k_neighbors=10)

columns = X.columns

os_data_X, os_data_y = os.fit_resample(X[:n], y)
os_data_X = pd.DataFrame(os_data_X, columns = columns)
os_data_y = pd.DataFrame(os_data_y, columns = ['is_canceled'])

In [151]:
n = len(os_data_X)

In [152]:
os_data_X = pd.concat([os_data_X, _X], axis=0)

In [153]:
scaler = StandardScaler()
X_st = scaler.fit_transform(os_data_X)

X_st 

array([[-0.33140481, -0.17114775,  0.06787613, ..., -0.06045333,
         0.60385473, -0.47052736],
       [-0.66144445, -1.58901535,  1.17906735, ..., -0.06045333,
         0.60385473, -0.47052736],
       [ 0.74122401,  1.24671985, -0.96923567, ..., -0.06045333,
         0.60385473, -0.47052736],
       ...,
       [-0.18472053,  1.24671985, -0.96923567, ..., -0.06045333,
         0.60385473, -0.47052736],
       [ 1.5388198 , -0.17114775,  0.73459087, ..., -0.06045333,
        -1.65602744,  2.12527494],
       [-0.79896097, -1.58901535,  1.47538501, ..., -0.06045333,
        -1.65602744,  2.12527494]])

In [154]:
len(_X)

23525

In [ ]:
%%time
scores = []
for i in range(1,15):
    score = {}
    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test) # метод predict возвращает вектор ответов для X_test
    
    score['k'] = i
    score['accuracy'] = accuracy_score(answers,y_test)
    score['recall'] = recall_score(answers,y_test)
    score['presicion'] = precision_score(answers,y_test)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

In [155]:
knn = KNeighborsClassifier(3)
knn.fit(os_data_X.iloc[:n], os_data_y)
answers = knn.predict(os_data_X.iloc[n:])

/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [156]:
len(answers)

23525

In [157]:
ans = pd.DataFrame(answers, columns=["is_canceled"])
ans = ans.reset_index()
ans

,index,is_canceled
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
23520,23520,1
23521,23521,0
23522,23522,0
23523,23523,1


In [160]:
ans.to_csv("res.csv", index=False)